In [ ]:
# We use the following line to allow us to embed plots in Jupyter notebooks
%matplotlib notebook

# Load libraries. We need tensorflow and numpy for training the CNN,
# matplotlib for plotting, os.path and PIL for loading images,
# and random to partition the data into test and training sets
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import math
from os.path import dirname, abspath
from os.path import join
import os.path
import PIL
import random
import urllib.request
from PIL import Image, ImageEnhance, ImageStat

#seed random for putting images into training and test arrays
random.seed(13456)

In [ ]:
#reverse lookup, output 0-8
r_lookup = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

#location of the weights/biases and architecture for the CNN checkpoint
model_checkpoint = "tmp/caffenet_pad_msh_tanzania.ckpt"

#location of the weights/biases and architecture for the CNN (saved above)
#model_for_identification = model_checkpoint

#create session
sess = tf.Session()

#load in the saved weights
new_saver = tf.train.import_meta_graph(model_checkpoint+'.meta')
new_saver.restore(sess, model_checkpoint) #tf.train.latest_checkpoint('tmp/./'))
#saver.restore(sess, model_for_identification)

#get graph to extract tensors
graph = tf.get_default_graph()
pred = graph.get_tensor_by_name("pred:0")
output = graph.get_tensor_by_name("output:0")
X = graph.get_tensor_by_name("X:0")
keep_prob = graph.get_tensor_by_name("keep_prob:0")


#called to identify subject in image
def identify(image):
    #reshape the image to an (1, 154587) vector
    image = np.mat(np.asarray(image).flatten())
    #find the prediction
    result = sess.run(pred, feed_dict={X: image, keep_prob: 1.})
    #we can look at the softmax output as well
    prob_array = sess.run(output, feed_dict={X: image, keep_prob: 1.})
    #print("op",prob_array)

    #return result (add 1 as indexed from 1 not 0)
    return result[0], prob_array

#setting path for saved file
path = "msh_tanzania_data/catagorize/"
#setting directory
dirs = os.listdir(path)

#looping over all the pictures in catagorize folder
for pics in dirs:
    #try:
    #open image to be categorized
    im = PIL.Image.open(path + pics)
    #identify image result
    predicted_drug_rlookup, predicted_prob = identify(im)
    
    #lookup
    predicted_drug = r_lookup[predicted_drug_rlookup]
    
    #load drug names
    #g = open('msh_tanzania_data/drug_labels.csv')
    #firstline = g.readline()
    #drugs = firstline.split(',')
    #print results to screen
    #print("Image:", image_name, "identified as drug", predicted_drug)# drugs[predicted_drug]
    
    write_to_file = join(str(predicted_drug) + "," + str(pics) + "\n")

    #print results to file.
    labelfile = open("categorize_results_reload.txt", "a+")
    labelfile.write(write_to_file)
    labelfile.close()

    #except:
    #print("Unable to write results to file for", pics)

